In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import re
import javalang

from batch_generators.java_batch_generator import JavaBatchGenerator

%load_ext autoreload
%autoreload 2

/home/sven/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_directory = '/home/sven/java_github_corpus'

In [22]:
tf.reset_default_graph()
a = tf.constant('  Hello World', dtype=tf.string)
with tf.Session() as sess:
    print(tf.size(tf.string_split([tf.py_func(lambda line: line.strip(), [a], tf.string)], '')).eval())
    print(tf.size(tf.string_split([a], '')).eval())

11
13


In [4]:
java_files = []

with open(os.path.join(data_directory, 'trainJava.csv'), 'r') as train_projects:
    for project in train_projects:
        if os.path.exists(os.path.join(data_directory, project.strip())):
            if len(java_files) > 100:
                break
            for subdir, _, files in os.walk(os.path.join(data_directory, project.strip())):
                for file in files:
                    java_files.append(os.path.join(data_directory, project.strip(), subdir, file))

print(len(java_files))

339


In [98]:
tf.reset_default_graph()
dataset = tf.data.Dataset.from_tensor_slices(java_files)

def map_function(line):
    def transform_string(s):
        s = s.strip()
        if random.random() > 0.9 and len(s) > 1:
            brackets = ['(', ')', '[', ']', '{', '}']
            bracket_indices = [i for i, c in enumerate(s) if c in brackets]
            if bracket_indices:
                drop_index = random.choice(bracket_indices)
                s = s[:drop_index] + s[drop_index+1:]
        if random.random() > 0.9 and len(s) > 1:
            semicolon_indices = [i for i, c in enumerate(s) if c == ';']
            if semicolon_indices:
                drop_index = random.choice(semicolon_indices)
                s = s[:drop_index] + s[drop_index+1:]
        if random.random() > 0.9 and len(s) > 1:
            change_char = random.randint(0, len(s)-2)
            s = s[:change_char] + s[change_char+1] + s[change_char] + s[drop_char+2:]
        return s
        
    t = tf.py_func(lambda string: string.strip(), [line], tf.string)
    t = tf.map_fn(lambda elem:
            tf.py_func(lambda char: np.array(ord(char), dtype=np.int32), [elem], tf.int32), tf.string_split([t], '').values, tf.int32)
    dec_inp = tf.concat([[0], t], 0)
    dec_out = tf.concat([t, [1000]], 0)
    enc_inp = tf.py_func(transform_string, [line], tf.string)
    enc_inp = tf.map_fn(lambda elem:
            tf.py_func(lambda char: np.array(ord(char), dtype=np.int32), [elem], tf.int32), tf.string_split([enc_inp], '').values, tf.int32)
    return enc_inp, tf.expand_dims(tf.size(enc_inp), 0), dec_inp, dec_out, tf.expand_dims(tf.size(dec_inp),0)

dataset = dataset.flat_map(
    lambda filename: (
        tf.data.TextLineDataset(filename)
        .filter(lambda line: 
                    tf.not_equal(
                        tf.size(tf.string_split([tf.py_func(lambda l: l.strip(), [line], tf.string)],"")), 
                        tf.constant(0, dtype=tf.int32))
               )
    )
)
dataset = dataset.map(map_function)
pad = tf.constant(128, dtype=tf.int32)
dataset = dataset.apply(tf.contrib.data.padded_batch_and_drop_remainder(2, padded_shapes=([None], [1], [None], [None], [1]), 
                               padding_values=(pad, tf.constant(0, dtype=tf.int32), pad, pad, tf.constant(0, dtype=tf.int32))))

iterator = dataset.make_one_shot_iterator()

a = iterator.get_next()

sess = tf.Session()
for i in range(25):
    z, b, c, d, e = sess.run(a)
    print(z)
    print(b)
    print(e)
    print(c)
    print(d)
    print("\n")
sess.close()

[[112  97  99 107  97 103 101  32 111 114 103  46 115 111 102 116 108  97
  110 103  46  99 108 105 101 110 116  46  99 111 109 112  97 110 121  73
  110 102 111  59]
 [105 109 112 111 114 116  32 106  97 118  97  46 105 111  46  83 101 114
  105  97 108 105 122  97  98 108 101  59 128 128 128 128 128 128 128 128
  128 128 128 128]]
[[40]
 [28]]
[[41]
 [29]]
[[  0 112  97  99 107  97 103 101  32 111 114 103  46 115 111 102 116 108
   97 110 103  46  99 108 105 101 110 116  46  99 111 109 112  97 110 121
   73 110 102 111  59]
 [  0 105 109 112 111 114 116  32 106  97 118  97  46 105 111  46  83 101
  114 105  97 108 105 122  97  98 108 101  59 128 128 128 128 128 128 128
  128 128 128 128 128]]
[[ 112   97   99  107   97  103  101   32  111  114  103   46  115  111
   102  116  108   97  110  103   46   99  108  105  101  110  116   46
    99  111  109  112   97  110  121   73  110  102  111   59 1000]
 [ 105  109  112  111  114  116   32  106   97  118   97   46  105  111
    46   83 

UnicodeDecodeError: 'utf8' codec can't decode byte 0xff in position 237: invalid start byte

In [164]:
a= tf.constant([[1, 2, 3], [4, 5, 6]])
with tf.Session() as sess:
    print(sess.run(tf.reverse(a, [1])))

[[3 2 1]
 [6 5 4]]


In [97]:
a = tf.constant([[1], [2], [3]])
with tf.Session() as sess:
    print(sess.run(tf.reshape(a,[3])))

[1 2 3]


In [149]:
a = tf.constant([1,2,3])
b = tf.random_uniform([1], minval = 0, maxval = tf.size(a))
with tf.Session() as sess:
    print(sess.run(tf.concat([tf.slice(a, [0], b), tf.slice(a, b+1, tf.subtract(tf.size(a), b+1))], 0)))

[2]


In [95]:
s = 'System.out.println(\"Hello World\");'
if random.random() > 0.9 and len(s) > 1:
    brackets = ['(', ')', '[', ']', '{', '}']
    bracket_indices = [i for i, c in s if c in brackets]
    if bracket_indices:
        drop_index = random.choice(bracket_indices)
        s = s[:drop_index] + s[drop_index+1:]
if random.random() > 0.9 and len(s) > 1:
    semicolon_indices = [i for i, c in s if c == ';']
    if semicolon_indices:
        drop_index = random.choice(semicolon_indices)
        s = s[:drop_index] + s[drop_index+1:]
if random.random() > 0.9 and len(s) > 1:
    drop_char = random.randint(0, len(s)-1)
    s = s[:drop_char] + s[drop_char+1:]
print(s)

ValueError: need more than 1 value to unpack

In [49]:
a = tf.constant(['     Hello', 'World    '], dtype=tf.string)
with tf.Session() as sess:
    print(sess.run(tf.map_fn(lambda line: tf.py_func(lambda string: [s.strip() for s in string], [line], tf.string), [a], tf.string)))

['Hello' 'World']


In [17]:
a = [1,2,3,4,5,6]
b = random.sample(a, 2)
print(b)

[3, 5]


In [12]:
for e in y:
    s = ''
    for f in e:
        s += chr(f)
    print(s)
print(z)

NameError: name 'y' is not defined

In [26]:
a = np.array([2**15], dtype='uint16')
b = np.unpackbits(a.view('uint8'))
print b

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]


In [11]:
tf.reset_default_graph()
target = tf.constant([[1,2,128,128],[3,4,5,128]], dtype=tf.int32)
pad = tf.constant(128, dtype=tf.int32)
b = tf.map_fn(lambda x: tf.map_fn(lambda y: tf.logical_not(tf.equal(y, pad)), x, dtype=tf.bool), target, dtype= tf.bool)
           
with tf.Session() as sess:
    result = tf.to_float(b).eval()
    print(result)
    print(target.eval())

[[1. 1. 0. 0.]
 [1. 1. 1. 0.]]
[[  1   2 128 128]
 [  3   4   5 128]]


In [8]:
tf.reset_default_graph()
a = tf.constant(["Hello World\nYou"], dtype=tf.string)
b = tf.string_split(a, delimiter='')

with tf.Session() as sess:
    c = b.eval()
    print(c)
    print(c.values)

SparseTensorValue(indices=array([[ 0,  0],
       [ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0,  6],
       [ 0,  7],
       [ 0,  8],
       [ 0,  9],
       [ 0, 10],
       [ 0, 11],
       [ 0, 12],
       [ 0, 13],
       [ 0, 14]]), values=array(['H', 'e', 'l', 'l', 'o', ' ', 'W', 'o', 'r', 'l', 'd', '\n', 'Y',
       'o', 'u'], dtype=object), dense_shape=array([ 1, 15]))
['H' 'e' 'l' 'l' 'o' ' ' 'W' 'o' 'r' 'l' 'd' '\n' 'Y' 'o' 'u']


In [8]:
tf.reset_default_graph()
batch_size = 128
batch_generator = 'Java' 
data_directory = data_directory
num_layers = 2
num_units = 256

encoder_input = tf.placeholder(tf.int32, shape=(batch_size, None, 1), name='encoder_input')
sequence_lengths = tf.placeholder(tf.int32, shape=(batch_size), name='sequence_lengths')
decoder_input = tf.placeholder(tf.int32, shape=(batch_size, None, 1), name='decoder_input')
target_output = tf.placeholder(tf.int32, shape=(batch_size, None), name='target_output')
target_lengths = tf.placeholder(tf.int32, shape=(batch_size), name="target_lengths")

pad_code = tf.constant(128, dtype = tf.int32)

target_weights = tf.to_int32(tf.map_fn(
                                lambda x: tf.map_fn(
                                            lambda y: tf.logical_not(tf.equal(y, pad_code)), 
                                            x, 
                                            dtype=tf.bool), 
                                target_output, 
                                dtype= tf.bool))

if batch_generator == "Java":
    batch_generator = JavaBatchGenerator(data_directory)
elif batch_generator == "Text":
    raise NotImplementedError("TextBatchGenerator is not implemented yet")
else:
    raise ValueError("batch_generator argument not recognized; must be one of: "
                     "Java, Text")

projection_layer = tf.layers.Dense(256, use_bias = False) # 256 characters can be represented in UTF-8

encoder_layers = [tf.nn.rnn_cell.LSTMCell(num_units) for i in range(num_layers)]
encoder_cell = tf.nn.rnn_cell.MultiRNNCell(encoder_layers)
encoder_outputs, encoder_state = tf.nn.dynamic_rnn(cell = encoder_cell,
                                                    inputs = tf.to_float(encoder_input),
                                                    sequence_length = sequence_lengths,
                                                    dtype = tf.float32)

decoder_layers = [tf.nn.rnn_cell.LSTMCell(num_units) for i in range(num_layers)]
decoder_cell = tf.nn.rnn_cell.MultiRNNCell(decoder_layers)
helper = tf.contrib.seq2seq.TrainingHelper(tf.to_float(decoder_input), target_lengths)
decoder = tf.contrib.seq2seq.BasicDecoder(
    decoder_cell, helper, encoder_state,
    output_layer=projection_layer)
outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder)
logits = outputs.rnn_output

crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=target_output, logits=logits)
train_loss = (tf.reduce_sum(crossent * tf.to_float(target_weights)) / batch_size)

tf.summary.FileWriter("/home/sven/logs", tf.get_default_graph()).close()

In [35]:
print(ord("\n"))

10
